<a href="https://colab.research.google.com/github/pratibhapradeep/Brain-Segmentation/blob/main/CV_project_brain_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Import necessary libraries
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, jaccard_score
import seaborn as sns
from datetime import datetime
import random

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set the correct path to your BraTS data
data_path = '/content/drive/MyDrive/original_data'
print(f"Data path: {data_path}")
print(f"Available directories: {sorted(os.listdir(data_path))}")

# Install required packages
!pip install nibabel scikit-image tqdm pandas matplotlib opencv-python seaborn

Mounted at /content/drive
Data path: /content/drive/MyDrive/original_data
Available directories: ['BraTS20_Processed_Training_001', 'BraTS20_Processed_Training_002', 'BraTS20_Processed_Training_003', 'BraTS20_Processed_Training_004', 'BraTS20_Processed_Training_005', 'BraTS20_Processed_Training_006', 'BraTS20_Processed_Training_007']


In [ ]:
# Define the Brain Tumor Segmentation Model based on Rastogi et al. paper
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

In [ ]:
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


In [ ]:
class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super(Up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        # Adjust dimensions if necessary
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = nn.functional.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])

        # Concatenate along the channel dimension
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [ ]:
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:
class BrainTumorSegmentationNet(nn.Module):
    def __init__(self, n_channels=4, n_classes=1, bilinear=True):
        super(BrainTumorSegmentationNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return torch.sigmoid(logits)

In [ ]:
# Loss functions
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, predicted, target):
        batch_size = predicted.size(0)

        # Flatten
        pred_flat = predicted.view(batch_size, -1)
        target_flat = target.view(batch_size, -1)

        # Calculate Dice
        intersection = (pred_flat * target_flat).sum(1)
        union = pred_flat.sum(1) + target_flat.sum(1)

        dice = (2. * intersection + self.smooth) / (union + self.smooth)

        return 1 - dice.mean()


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.8, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce_loss = nn.BCELoss(reduction='none')

    def forward(self, predicted, target):
        bce = self.bce_loss(predicted, target)

        # Apply focal weighting
        pt = torch.exp(-bce)
        focal_weight = (1 - pt) ** self.gamma

        # Apply alpha for class imbalance
        if self.alpha is not None:
            focal_weight = self.alpha * focal_weight

        return (focal_weight * bce).mean()

In [ ]:
class CombinedLoss(nn.Module):
    def __init__(self, dice_weight=0.5, focal_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
        self.dice_loss = DiceLoss()
        self.focal_loss = FocalLoss()

    def forward(self, predicted, target):
        dice = self.dice_loss(predicted, target)
        focal = self.focal_loss(predicted, target)
        return self.dice_weight * dice + self.focal_weight * focal

In [ ]:
# Helper function for calculating Dice coefficient
def dice_coefficient(pred, target):
    smooth = 1.0

    pred_flat = pred.view(-1)
    target_flat = target.view(-1)

    intersection = (pred_flat * target_flat).sum()
    union = pred_flat.sum() + target_flat.sum()

    return (2. * intersection + smooth) / (union + smooth)

In [ ]:
# Helper function for calculating IoU (Jaccard index)
def iou_coefficient(pred, target):
    smooth = 1.0

    pred_flat = pred.view(-1)
    target_flat = target.view(-1)

    intersection = (pred_flat * target_flat).sum()
    union = pred_flat.sum() + target_flat.sum() - intersection

    return (intersection + smooth) / (union + smooth)

In [ ]:
# Custom Dataset for BraTS
class BraTSDataset(Dataset):
    def __init__(self, root_dir, slice_range=(30, 120), slice_step=3):
        self.root_dir = root_dir
        self.slice_range = slice_range
        self.slice_step = slice_step
        self.samples = []

        # Get all directories (each contains one patient data)
        self.folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f)) and "Training" in f]

        print(f"Found {len(self.folders)} training folders")

        # Explore each folder to find the MRI modalities
        for folder in tqdm(self.folders, desc="Scanning folders"):
            folder_path = os.path.join(root_dir, folder)

            # Find the files for each modality
            t1_file = None
            t1ce_file = None
            t2_file = None
            flair_file = None
            seg_file = None

            # Search for the MRI files in each patient folder
            for file in os.listdir(folder_path):
                if file.endswith('.nii.gz') or file.endswith('.nii'):
                    if 't1.' in file.lower() or '_t1_' in file.lower() or '_t1.' in file.lower():
                        t1_file = os.path.join(folder_path, file)
                    elif 't1ce' in file.lower() or 't1c.' in file.lower() or 't1_ce' in file.lower():
                        t1ce_file = os.path.join(folder_path, file)
                    elif 't2.' in file.lower() or '_t2_' in file.lower() or '_t2.' in file.lower():
                        t2_file = os.path.join(folder_path, file)
                    elif 'flair' in file.lower():
                        flair_file = os.path.join(folder_path, file)
                    elif 'seg' in file.lower() or 'mask' in file.lower():
                        seg_file = os.path.join(folder_path, file)

            # Check if we found all modalities
            if all([t1_file, t1ce_file, t2_file, flair_file, seg_file]):
                # Load and get dimensions
                img = nib.load(t1_file)
                n_slices = img.shape[2]  # Number of slices in the 3D volume

                # Select slices in the specified range with the given step
                min_slice = max(0, slice_range[0])
                max_slice = min(n_slices, slice_range[1])

                for slice_idx in range(min_slice, max_slice, slice_step):
                    self.samples.append({
                        'folder': folder,
                        'slice_idx': slice_idx,
                        't1_file': t1_file,
                        't1ce_file': t1ce_file,
                        't2_file': t2_file,
                        'flair_file': flair_file,
                        'seg_file': seg_file
                    })
            else:
                missing = []
                if not t1_file: missing.append("T1")
                if not t1ce_file: missing.append("T1ce")
                if not t2_file: missing.append("T2")
                if not flair_file: missing.append("FLAIR")
                if not seg_file: missing.append("Segmentation")
                print(f"Missing modalities in {folder}: {', '.join(missing)}")

        print(f"Created dataset with {len(self.samples)} slices from {len(self.folders)} folders")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Load each modality
        t1_slice = self._load_slice(sample['t1_file'], sample['slice_idx'])
        t1ce_slice = self._load_slice(sample['t1ce_file'], sample['slice_idx'])
        t2_slice = self._load_slice(sample['t2_file'], sample['slice_idx'])
        flair_slice = self._load_slice(sample['flair_file'], sample['slice_idx'])

        # Load segmentation mask
        seg_slice = self._load_slice(sample['seg_file'], sample['slice_idx'], is_mask=True)

        # Combine all modalities into a 4-channel input
        image = np.stack([t1_slice, t1ce_slice, t2_slice, flair_slice], axis=0)

        # Convert to PyTorch tensors
        image_tensor = torch.from_numpy(image).float()
        mask_tensor = torch.from_numpy(seg_slice).float().unsqueeze(0)  # Add channel dimension

        return {
            'image': image_tensor,
            'mask': mask_tensor,
            'folder': sample['folder'],
            'slice_idx': sample['slice_idx']
        }

    def _load_slice(self, file_path, slice_idx, is_mask=False):
        """Load a single slice from a NIfTI file and normalize"""
        img = nib.load(file_path)
        data = img.get_fdata()
        slice_data = data[:, :, slice_idx]

        if is_mask:
            # Binary mask: any non-zero value is considered tumor
            return (slice_data > 0).astype(np.float32)
        else:
            # Normalize image to [0,1]
            min_val = np.min(slice_data)
            max_val = np.max(slice_data)

            if max_val > min_val:
                normalized = (slice_data - min_val) / (max_val - min_val)
                return normalized
            else:
                return np.zeros_like(slice_data, dtype=np.float32)

In [ ]:
# Training function
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0

    with tqdm(dataloader, desc="Training") as pbar:
        for batch in pbar:
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Calculate loss
            loss = criterion(outputs, masks)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Update metrics
            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

    return epoch_loss / len(dataloader)

In [ ]:
# Validation function with detailed metrics
def validate(model, dataloader, criterion, device):
    model.eval()
    val_loss = 0
    val_dice = 0
    val_iou = 0
    val_precision = 0
    val_recall = 0
    val_f1 = 0

    all_preds = []
    all_targets = []

    with torch.no_grad():
        with tqdm(dataloader, desc="Validation") as pbar:
            for batch in pbar:
                images = batch['image'].to(device)
                masks = batch['mask'].to(device)

                # Forward pass
                outputs = model(images)

                # Calculate loss
                loss = criterion(outputs, masks)
                val_loss += loss.item()

                # Calculate metrics
                pred_masks = (outputs > 0.5).float()

                # Dice coefficient
                dice = dice_coefficient(pred_masks, masks)
                val_dice += dice.item()

                # IoU (Jaccard index)
                iou = iou_coefficient(pred_masks, masks)
                val_iou += iou.item()

                # Move to CPU for sklearn metrics
                pred_np = pred_masks.cpu().numpy().flatten()
                target_np = masks.cpu().numpy().flatten()

                # Store for confusion matrix
                all_preds.extend(pred_np)
                all_targets.extend(target_np)

                # Calculate precision, recall, F1
                if np.sum(target_np) > 0:  # Skip slices with no tumor
                    val_precision += precision_score(target_np, pred_np, zero_division=1)
                    val_recall += recall_score(target_np, pred_np, zero_division=1)
                    val_f1 += f1_score(target_np, pred_np, zero_division=1)

                pbar.set_postfix(loss=loss.item(), dice=dice.item())

    # Compute confusion matrix
    cm = confusion_matrix(all_targets, all_preds)

    # Calculate average metrics
    metrics = {
        'loss': val_loss / len(dataloader),
        'dice': val_dice / len(dataloader),
        'iou': val_iou / len(dataloader),
        'precision': val_precision / len(dataloader),
        'recall': val_recall / len(dataloader),
        'f1': val_f1 / len(dataloader),
        'confusion_matrix': cm
    }

    return metrics

In [ ]:
# Visualization function with multiple visualizations
def visualize_predictions(model, val_loader, device, num_samples=5):
    model.eval()

    # Get a batch
    batch = next(iter(val_loader))
    images = batch['image'].to(device)
    masks = batch['mask'].to(device)

    # Get predictions
    with torch.no_grad():
        outputs = model(images)
        preds = (outputs > 0.5).float()

    # Convert to numpy
    images_np = images.cpu().numpy()
    masks_np = masks.cpu().numpy()
    outputs_np = outputs.cpu().numpy()
    preds_np = preds.cpu().numpy()

    # 1. Basic prediction visualization
    fig1, axes = plt.subplots(min(num_samples, len(images)), 5, figsize=(20, 4*min(num_samples, len(images))))

    for i in range(min(num_samples, len(images))):
        # T1 image
        axes[i, 0].imshow(images_np[i, 0], cmap='gray')
        axes[i, 0].set_title("T1")
        axes[i, 0].axis('off')

        # FLAIR image
        axes[i, 1].imshow(images_np[i, 3], cmap='gray')
        axes[i, 1].set_title("FLAIR")
        axes[i, 1].axis('off')

        # Ground truth
        axes[i, 2].imshow(masks_np[i, 0], cmap='gray')
        axes[i, 2].set_title("Ground Truth")
        axes[i, 2].axis('off')

        # Probability map
        im = axes[i, 3].imshow(outputs_np[i, 0], cmap='jet', vmin=0, vmax=1)
        axes[i, 3].set_title("Probability")
        axes[i, 3].axis('off')
        plt.colorbar(im, ax=axes[i, 3], fraction=0.046, pad=0.04)

        # Binary prediction
        axes[i, 4].imshow(preds_np[i, 0], cmap='gray')
        axes[i, 4].set_title("Prediction")
        axes[i, 4].axis('off')

    plt.tight_layout()

    # 2. Overlay visualization
    fig2, axes = plt.subplots(min(num_samples, len(images)), 3, figsize=(15, 4*min(num_samples, len(images))))

    for i in range(min(num_samples, len(images))):
        # Original FLAIR image
        axes[i, 0].imshow(images_np[i, 3], cmap='gray')
        axes[i, 0].set_title("FLAIR Image")
        axes[i, 0].axis('off')

        # Ground truth overlay
        axes[i, 1].imshow(images_np[i, 3], cmap='gray')
        mask_overlay = np.ma.masked_where(masks_np[i, 0] < 0.5, masks_np[i, 0])
        axes[i, 1].imshow(mask_overlay, cmap='autumn', alpha=0.7)
        axes[i, 1].set_title("Ground Truth Overlay")
        axes[i, 1].axis('off')

        # Prediction overlay
        axes[i, 2].imshow(images_np[i, 3], cmap='gray')
        pred_overlay = np.ma.masked_where(preds_np[i, 0] < 0.5, preds_np[i, 0])
        axes[i, 2].imshow(pred_overlay, cmap='cool', alpha=0.7)
        axes[i, 2].set_title("Prediction Overlay")
        axes[i, 2].axis('off')

    plt.tight_layout()

    # 3. Error analysis visualization
    fig3, axes = plt.subplots(min(num_samples, len(images)), 3, figsize=(15, 4*min(num_samples, len(images))))

    for i in range(min(num_samples, len(images))):
        # Original FLAIR image
        axes[i, 0].imshow(images_np[i, 3], cmap='gray')
        axes[i, 0].set_title("FLAIR Image")
        axes[i, 0].axis('off')

        # True Positive (green) and False Negative (red)
        tp = np.logical_and(preds_np[i, 0] > 0.5, masks_np[i, 0] > 0.5)
        fn = np.logical_and(preds_np[i, 0] <= 0.5, masks_np[i, 0] > 0.5)

        axes[i, 1].imshow(images_np[i, 3], cmap='gray')
        axes[i, 1].imshow(np.ma.masked_where(tp == 0, tp), cmap='Greens', alpha=0.7)
        axes[i, 1].imshow(np.ma.masked_where(fn == 0, fn), cmap='Reds', alpha=0.7)
        axes[i, 1].set_title("TP (green) & FN (red)")
        axes[i, 1].axis('off')

        # True Negative (black) and False Positive (blue)
        tn = np.logical_and(preds_np[i, 0] <= 0.5, masks_np[i, 0] <= 0.5)
        fp = np.logical_and(preds_np[i, 0] > 0.5, masks_np[i, 0] <= 0.5)

        axes[i, 2].imshow(images_np[i, 3], cmap='gray')
        # We don't visualize true negatives as they're the majority of the image
        axes[i, 2].imshow(np.ma.masked_where(fp == 0, fp), cmap='Blues', alpha=0.7)
        axes[i, 2].set_title("FP (blue)")
        axes[i, 2].axis('off')

    plt.tight_layout()

    return fig1, fig2, fig3


In [ ]:
# Visualize training curves with multiple metrics
def plot_training_curves(metrics_df, output_dir):
    # 1. Loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(metrics_df['epoch'], metrics_df['train_loss'], 'b-', label='Training Loss')
    plt.plot(metrics_df['epoch'], metrics_df['val_loss'], 'r-', label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'loss_curves.png'))
    plt.close()

    # 2. Dice and IoU curves
    plt.figure(figsize=(10, 6))
    plt.plot(metrics_df['epoch'], metrics_df['val_dice'], 'g-', label='Dice Coefficient')
    plt.plot(metrics_df['epoch'], metrics_df['val_iou'], 'y-', label='IoU (Jaccard)')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.title('Validation Dice and IoU Scores')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'dice_iou_curves.png'))
    plt.close()

    # 3. Precision, Recall, F1 curves
    plt.figure(figsize=(10, 6))
    plt.plot(metrics_df['epoch'], metrics_df['val_precision'], 'm-', label='Precision')
    plt.plot(metrics_df['epoch'], metrics_df['val_recall'], 'c-', label='Recall')
    plt.plot(metrics_df['epoch'], metrics_df['val_f1'], 'k-', label='F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.title('Validation Precision, Recall, and F1 Scores')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'precision_recall_f1_curves.png'))
    plt.close()

# Plot confusion matrix
def plot_confusion_matrix(cm, output_dir):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues',
                xticklabels=['No Tumor', 'Tumor'],
                yticklabels=['No Tumor', 'Tumor'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
    plt.close()

In [ ]:
# Create data loaders
def get_data_loaders(root_dir, batch_size=8, val_split=0.2, slice_range=(30, 120), slice_step=3, num_workers=2):
    # Create dataset
    dataset = BraTSDataset(root_dir, slice_range=slice_range, slice_step=slice_step)

    # Check if dataset is empty
    if len(dataset) == 0:
        raise ValueError(f"No valid data found in {root_dir}. Check paths and file names.")

    # Split into train/val datasets
    train_size = int((1 - val_split) * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42)
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )

    return train_loader, val_loader



In [ ]:
# Main training function with different optimizers
def train_model(data_path, output_dir='model_output', batch_size=4, num_epochs=30,
               slice_step=3, optimizer_name='adam', learning_rate=1e-4, loss_type='dice'):
    # Create experiment directory with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    exp_name = f"{optimizer_name}_lr{learning_rate}_{loss_type}_{timestamp}"
    output_dir = os.path.join(output_dir, exp_name)
    os.makedirs(output_dir, exist_ok=True)

    # Set device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Get data loaders
    train_loader, val_loader = get_data_loaders(
        data_path,
        batch_size=batch_size,
        slice_step=slice_step
    )

    print(f"Training set: {len(train_loader.dataset)} slices")
    print(f"Validation set: {len(val_loader.dataset)} slices")

    # Initialize model
    model = BrainTumorSegmentationNet(n_channels=4, n_classes=1)
    model = model.to(device)

    # Initialize loss function based on type
    if loss_type == 'dice':
        criterion = DiceLoss()
    elif loss_type == 'focal':
        criterion = FocalLoss()
    elif loss_type == 'combined':
        criterion = CombinedLoss()
    else:
        criterion = DiceLoss()  # Default

    # Initialize optimizer based on name
    if optimizer_name == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    elif optimizer_name == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    else:
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Default

    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    # Track metrics
    train_losses = []
    val_metrics_list = []
    best_val_dice = 0.0
    best_epoch = 0

     # Save configuration
    config = {
        'data_path': data_path,
        'optimizer': optimizer_name,
        'learning_rate': learning_rate,
        'loss_function': loss_type,
        'batch_size': batch_size,
        'epochs': num_epochs,
        'slice_step': slice_step,
        'device': str(device),
        'timestamp': timestamp
    }

    # Save config to JSON
    import json
    with open(os.path.join(output_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)

    # Training loop
    start_time = time.time()
    metrics_df = pd.DataFrame()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        # Train
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)

        # Validate with detailed metrics
        val_metrics = validate(model, val_loader, criterion, device)

        # Print metrics
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_metrics['loss']:.4f} | "
              f"Val Dice: {val_metrics['dice']:.4f} | Val IoU: {val_metrics['iou']:.4f}")

        # Save metrics
        train_losses.append(train_loss)
        val_metrics_list.append(val_metrics)

        # Update learning rate based on validation loss
        scheduler.step(val_metrics['loss'])

        # Save best model
        if val_metrics['dice'] > best_val_dice:
            best_val_dice = val_metrics['dice']
            best_epoch = epoch + 1
            torch.save(model.state_dict(), os.path.join(output_dir, 'best_model.pth'))
            print(f"Saved new best model with Dice: {val_metrics['dice']:.4f}")

            # Save confusion matrix for best model
            plot_confusion_matrix(val_metrics['confusion_matrix'], output_dir)

        # Save checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_metrics': val_metrics
            }, os.path.join(output_dir, f'checkpoint_epoch_{epoch+1}.pth'))

        # Collect all metrics for DataFrame
        epoch_metrics = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_metrics['loss'],
            'val_dice': val_metrics['dice'],
            'val_iou': val_metrics['iou'],
            'val_precision': val_metrics['precision'],
            'val_recall': val_metrics['recall'],
            'val_f1': val_metrics['f1'],
            'learning_rate': optimizer.param_groups[0]['lr']
        }

        # Append to DataFrame
        metrics_df = pd.concat([metrics_df, pd.DataFrame([epoch_metrics])], ignore_index=True)

    # Save final model
    torch.save(model.state_dict(), os.path.join(output_dir, 'final_model.pth'))

    # Training time
    training_time = time.time() - start_time
    print(f"\nTraining completed in {training_time / 60:.2f} minutes")
    print(f"Best validation Dice: {best_val_dice:.4f} at epoch {best_epoch}")

    # Save metrics to CSV
    metrics_df.to_csv(os.path.join(output_dir, 'training_metrics.csv'), index=False)

    # Plot training curves
    plot_training_curves(metrics_df, output_dir)

    # Visualize predictions
    print("Generating visualizations...")
    model.load_state_dict(torch.load(os.path.join(output_dir, 'best_model.pth')))
    fig1, fig2, fig3 = visualize_predictions(model, val_loader, device)
    fig1.savefig(os.path.join(output_dir, 'basic_predictions.png'))
    fig2.savefig(os.path.join(output_dir, 'overlay_predictions.png'))
    fig3.savefig(os.path.join(output_dir, 'error_analysis.png'))

    plt.close(fig1)
    plt.close(fig2)
    plt.close(fig3)

    # Generate detailed test report
    test_report = {
        'experiment_name': exp_name,
        'best_epoch': best_epoch,
        'best_dice': best_val_dice,
        'final_metrics': {k: v for k, v in val_metrics.items() if k != 'confusion_matrix'},
        'training_time_minutes': training_time / 60,
        'config': config
    }

    with open(os.path.join(output_dir, 'test_report.json'), 'w') as f:
        json.dump(test_report, f, indent=4)

    return model, metrics_df, output_dir



In [ ]:
# Function to run multiple experiments with different configurations
def run_experiments(data_path, base_output_dir):
    # Define experiment configurations
    experiments = [
        # Different optimizers
        {'optimizer_name': 'adam', 'learning_rate': 1e-4, 'loss_type': 'dice', 'num_epochs': 10},
        {'optimizer_name': 'sgd', 'learning_rate': 1e-3, 'loss_type': 'dice', 'num_epochs': 10},
        {'optimizer_name': 'rmsprop', 'learning_rate': 5e-4, 'loss_type': 'dice', 'num_epochs': 10},

        # Different loss functions
        {'optimizer_name': 'adam', 'learning_rate': 1e-4, 'loss_type': 'focal', 'num_epochs': 10},
        {'optimizer_name': 'adam', 'learning_rate': 1e-4, 'loss_type': 'combined', 'num_epochs': 10},

        # Different learning rates
        {'optimizer_name': 'adam', 'learning_rate': 5e-5, 'loss_type': 'dice', 'num_epochs': 10},
        {'optimizer_name': 'adam', 'learning_rate': 5e-4, 'loss_type': 'dice', 'num_epochs': 10},
    ]

    # Create results directory
    results_dir = os.path.join(base_output_dir, f'experiments_{datetime.now().strftime("%Y%m%d")}')
    os.makedirs(results_dir, exist_ok=True)

    # Run each experiment
    experiment_results = []

    for i, exp_config in enumerate(experiments):
        print(f"\n\n{'='*80}")
        print(f"Running experiment {i+1}/{len(experiments)}: {exp_config}")
        print(f"{'='*80}\n")

        # Train model with this configuration
        model, metrics, output_dir = train_model(
            data_path=data_path,
            output_dir=results_dir,
            batch_size=4,
            **exp_config
        )

        # Store results
        best_dice = metrics['val_dice'].max()
        best_epoch = metrics['val_dice'].argmax() + 1

        experiment_results.append({
            'experiment_id': i+1,
            'config': exp_config,
            'best_dice': best_dice,
            'best_epoch': best_epoch,
            'output_dir': output_dir
        })

        print(f"\nExperiment {i+1} completed with best Dice: {best_dice:.4f} at epoch {best_epoch}")

    # Create comparison report
    comparison_df = pd.DataFrame([
        {
            'Experiment': i+1,
            'Optimizer': exp['config']['optimizer_name'],
            'Learning Rate': exp['config']['learning_rate'],
            'Loss Function': exp['config']['loss_type'],
            'Best Dice': exp['best_dice'],
            'Best Epoch': exp['best_epoch']
        }
        for i, exp in enumerate(experiment_results)
    ])

    # Save comparison report
    comparison_df.to_csv(os.path.join(results_dir, 'experiment_comparison.csv'), index=False)

    # Plot comparison bar chart
    plt.figure(figsize=(12, 6))

    # Create experiment labels
    exp_labels = [
        f"{exp['config']['optimizer_name']}\n"
        f"lr={exp['config']['learning_rate']}\n"
        f"{exp['config']['loss_type']}"
        for exp in experiment_results
    ]

    # Plot Dice scores
    dice_scores = [exp['best_dice'] for exp in experiment_results]
    bars = plt.bar(exp_labels, dice_scores, color='skyblue')

    # Add value labels
    for bar, score in zip(bars, dice_scores):
        plt.text(
            bar.get_x() + bar.get_width()/2,
            bar.get_height() + 0.01,
            f"{score:.4f}",
            ha='center'
        )

    plt.ylabel('Best Dice Coefficient')
    plt.title('Comparison of Experiment Configurations')
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    plt.savefig(os.path.join(results_dir, 'experiment_comparison.png'))
    plt.close()

    print(f"\nAll experiments completed! Results saved to {results_dir}")
    return comparison_df, experiment_results


In [ ]:
# Run the code with either a single model or multiple experiments
if __name__ == "__main__":
    # Set parameters
    batch_size = 4          # Adjust based on GPU memory
    num_epochs = 10         # Default epochs for training
    slice_step = 3          # Use every third slice to save memory

    """
    Either train a single model...
    model, metrics_df, output_dir = train_model(
        data_path=data_path,  # This is set at the top of the script
        output_dir='/content/drive/MyDrive/brain_tumor_model_output',
        batch_size=batch_size,
        num_epochs=num_epochs,
        slice_step=slice_step,
        optimizer_name='adam',  # 'adam', 'sgd', or 'rmsprop'
        learning_rate=1e-4,
        loss_type='dice'        # 'dice', 'focal', or 'combined'
    )

    print(f"Single model training complete! Results saved to {output_dir}")

    """
    # Or uncomment to run multiple experiments

    comparison_df, experiment_results = run_experiments(
        data_path=data_path,
        base_output_dir='/content/drive/MyDrive/brain_tumor_model_output'
    )

    print("Multiple experiments complete!")




Running experiment 1/7: {'optimizer_name': 'adam', 'learning_rate': 0.0001, 'loss_type': 'dice', 'num_epochs': 10}

Using device: cpu
Found 7 training folders


Scanning folders: 100%|██████████| 7/7 [00:00<00:00, 105.33it/s]

Created dataset with 210 slices from 7 folders
Training set: 168 slices
Validation set: 42 slices



/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch 1/10


...

Train Loss: 0.6803 | Val Loss: 0.7844 | Val Dice: 0.4643 | Val IoU: 0.3539

Epoch 9/10


Validation: 100%|██████████| 11/11 [00:17<00:00,  1.55s/it, dice=0.000833, loss=0.996]


Train Loss: 0.6882 | Val Loss: 0.7380 | Val Dice: 0.4681 | Val IoU: 0.3528

Epoch 10/10


Validation: 100%|██████████| 11/11 [00:17<00:00,  1.60s/it, dice=0.000252, loss=0.999]


Train Loss: 0.6820 | Val Loss: 0.7555 | Val Dice: 0.3662 | Val IoU: 0.2534

Training completed in 38.09 minutes
Best validation Dice: 0.6087 at epoch 5
Generating visualizations...

Experiment 7 completed with best Dice: 0.6087 at epoch 5

All experiments completed! Results saved to /content/drive/MyDrive/brain_tumor_model_output/experiments_20250425
Multiple experiments complete!
